## Setting Up LMCache with Gemma

In [ ]:
import torch

# Verify GPU
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️  No GPU available, using CPU")

In [ ]:
import os
from pathlib import Path

# Create cache directory
cache_dir = Path("/content/lmcache_warm")
cache_dir.mkdir(parents=True, exist_ok=True)

print(f"✅ Cache directory: {cache_dir}")
print(f"   Available space: {os.statvfs(cache_dir).f_bavail * os.statvfs(cache_dir).f_frsize / 1e9:.1f} GB")

In [ ]:
#TODO: change to fit with fakeredis
%%bash
# Install via APT
apt-get update -qq
apt-get install -y redis-server > /dev/null 2>&1

# Start as daemon
redis-server --daemonize yes --port 6379

# Verify
redis-cli ping  # Should return PONG

In [ ]:
# Verify Redis connection
import fakeredis

try:
    r = redis.Redis(host='localhost', port=6379, db=0)
    r.ping()
    print("✅ Redis is running")
    
    # Check info
    info = r.info('server')
    print(f"   Version: {info['redis_version']}")
    print(f"   Mode: {info['redis_mode']}")
except redis.ConnectionError:
    print("❌ Redis not running. Run installation cell first.")

In [ ]:
cache = ThreeTierCache(
    hot_size_gb=1,      # 1GB GPU (T4 has 15GB total)
    warm_ram_gb=3,      # 3GB RAM (leave room for Colab system)
    warm_disk_gb=10,    # 10GB disk
)

In [ ]:
# Hot + Warm: Local (fast, temporary)
# Cold: Export to Drive periodically

import shutil

def backup_redis_to_drive():
    """Backup Redis data to Google Drive"""
    r = redis.Redis()
    keys = r.keys('lmcache:*')
    
    backup_dir = Path("/content/drive/MyDrive/redis_backup")
    backup_dir.mkdir(exist_ok=True)
    
    for key in keys:
        data = r.get(key)
        with open(backup_dir / f"{key.decode()}.pkl", 'wb') as f:
            f.write(data)
    
    print(f"✅ Backed up {len(keys)} keys to Drive")

# Call periodically
backup_redis_to_drive()